## Agents perform better when role-playing
- You should focus on goals and expectations
- One Agent can do multiple tasks and use multiple tools ( but limit it)
- Tasks and Agents should be granular
- Tasks can be executed in different ways
- It's pretty easy to create multi-agents with crew AI
- eg: You: give me an analysis on tesla stock vs you are a FINRA approved financial analyst. give me an analysis on tesla stock
- Other charectistics of Agents are,
  - Cooperation, Focus, Guardrails, Tools, 
  - Memory - 3 types. Short term, Long Term ( kept evn after Agent finishes ) and Entity Memory ( Stores person, Org, locations)

In [1]:
# Warning control
import json, os
import warnings
warnings.filterwarnings('ignore')
os.environ["CHROMA_TELEMETRY"] = "false"

In [2]:
from __future__ import annotations
from crewai import Agent, Task, Crew, LLM, Process
from crewai_tools import FileReadTool, TavilySearchTool, ScrapeWebsiteTool, JSONSearchTool

- As a LLM for your agents, you'll be using OpenAI's gpt-3.5-turbo.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the bottom of the notebook.

In [3]:
from typing import Any, Dict, Type
from pydantic import BaseModel, Field, validator
from crewai.tools import BaseTool


class SaveJSONArgs(BaseModel):
    path: str = Field(..., description="Output file path (e.g., ./newsletter.json).")
    payload: Dict[str, Any] = Field(..., description="Final newsletter JSON payload.")

    @validator("path")
    def ensure_parent_dir(cls, v: str) -> str:
        d = os.path.dirname(v) or "."
        os.makedirs(d, exist_ok=True)
        return v


class SaveJSONTool(BaseTool):
    name: str = "save_json"
    description: str = "Persist a given JSON-serializable payload to disk."
    args_schema: Type[BaseModel] = SaveJSONArgs  # Add type annotation here

    def _run(self, path: str, payload: Dict[str, Any]) -> str:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(payload, f, ensure_ascii=False, indent=2)
        return f"Saved newsletter to {path}"

In [4]:
# Initialize the tool with the path to your JSON file
file_read_tool = FileReadTool(file_path='./files/tavily_response.json')
save_json = SaveJSONTool()

In [5]:
def read_prompt(filename: str) -> str:
    """
    Reads the content of a markdown (.md) file from the ./file directory.

    Args:
        filename (str): The name of the markdown file (e.g., 'example.md').

    Returns:
        str: The content of the markdown file.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the filename does not end with '.md'.
        Exception: For other unexpected errors.
    """

    if not filename.endswith('.md'):
        raise ValueError("Provided file must have a '.md' extension.")

    file_path = os.path.join('./files', filename)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"File '{file_path}' not found.")
    except Exception as e:
        raise Exception(f"Error reading file '{file_path}': {e}")

In [6]:
# --- Agents ---
orchestrator = Agent(
    role="Newsletter Orchestrator",
    goal=(
        "Read the JSON via FileReadTool, pick the Main news Article, derive exactly 5 subtopics"
        "and bucket up to 5 news articles per subtopic; ignore outliers; then assemble final output."
    ),
    backstory=(
        "A meticulous news curator: balances score-based selection with judgment; clusters semantically "
        "coherent themes; produces clean, enforceable structure."
    ),
    tools=[file_read_tool, save_json],
    allow_delegation=False,
    verbose=True,
    max_iter=3,                 # keeps the agent from looping
    max_rpm=20,
    max_execution_time=120
)

In [7]:
writer = Agent(
    role="Newsletter Writer",
    goal="Draft engaging, accurate editorials for each subtopic, then synthesize a main editorial.",
    backstory=(
        "A seasoned Healtcare technology news writer who turns multiple sources into crisp, trustworthy prose."
    ),
    allow_delegation=False,
    verbose=True,
)

In [8]:
editor = Agent(
    role="Newsletter Editor",
    goal="Refine drafts for clarity, tone, coherence, and fidelity to sources.",
    backstory="A veteran editor ensuring publication-ready copy while preserving source alignment.",
    allow_delegation=False,
    verbose=True,
)

In [9]:
# --- Tasks ---

outline_task_description = read_prompt("outline_task.md") 

# 1) Outline: use FileReadTool to extract EXACTLY your fields from results[].
outline_task = Task(
    description=outline_task_description,
    expected_output="JSON with items[], main_idx, main_rationale, and 1–5 subtopics with idxs.",
    agent=orchestrator,
    tools=[file_read_tool],
)

In [10]:
# 2) Writing: subtopic editorials first, then main editorial.
write_task_description = read_prompt("write_task.md")

write_task = Task(
    description=write_task_description,
    expected_output="JSON with subtopic_editorials[] and main_editorial.",
    agent=writer,
)

In [11]:
# 3) Editing: refine drafts, keep same JSON shape, preserve sources.
edit_task = Task(
    description=(
        "Refine the Writer JSON for clarity, tone consistency, and factual alignment with the provided sources. "
        "Keep roughly the same lengths; DO NOT remove sources. Output the SAME JSON SHAPE."
    ),
    expected_output="Edited JSON with subtopic_editorials[] and main_editorial (same shape).",
    agent=editor,
)

In [12]:
save_task_description = read_prompt("save_task.md")

# 4) Assemble & Save: merge outline + edited copy + items, save final JSON.
save_task = Task(
    description=save_task_description,
    expected_output="Confirmation string from save_json that the file is saved.",
    agent=orchestrator,
    tools=[save_json],
)

In [13]:
crew = Crew(
    agents=[orchestrator, writer, editor],
    tasks=[outline_task, write_task, edit_task, save_task],
    process=Process.sequential,
    verbose=True,
)


In [14]:
if __name__ == "__main__":
    result = crew.kickoff(inputs={
        "output_path": "./output/newsletter.json"
    })
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 68999b2a-0f1c-4867-9d88-3150148c6820                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Newsletter Orchestrator                                                                                 │
│                                                                                                                 │
│  Task: [C] CONTEXT                                                                                              │
│  You have access to a single tool named exactly: Read a file's content                                          │
│  Its arguments are:                                                                                             │
│  - file_path: str (mandatory, full path of the file to read)                                                    │
│  - start_line: int | null (optional; use null to read from the beginning)                                       │
│  - line_count: int | null (optional; use null to read to the end)                                               │
│                                                                                                                 │
│  The input file is a JSON document with the schema:                                                             │
│  {                                                                                                              │
│    "query": string | null,                                                                                      │
│    "follow_up_questions": any | null,                                                                           │
│    "answer": any | null,                                                                                        │
│    "images": array,                                                                                             │
│    "results": [                                                                                                 │
│      {                                                                                                          │
│        "url": string,                                                                                           │
│        "title": string | null,                                                                                  │
│        "content": string | null,                                                                                │
│        "score": number | null,                                                                                  │
│        "raw_content": string | null                                                                             │
│      },                                                                                                         │
│      ...                                                                                                        │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│  You must perform a single pass: read → parse → normalize → summarize → select main → derive subtopics → emit   │
│  JSON → stop.                                                                                                   │
│                                                                                                                 │
│  [O] OBJECTIVE                                                                                                  │
│  Produce a normalized, self-contained JSON object that:                                                         │
│  1) Extracts, cleans, and summarizes the items from res

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Newsletter Orchestrator                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I will read the JSON file to extract and clean the news articles, select the Main Article    │
│  based on the highest score, derive 5 subtopics, and finally emit the required JSON format.                     │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"./files/tavily_response.json\"}"                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Newsletter Orchestrator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Confirmation string from save_json that the file is saved.                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8c44984e-347d-4fbe-b8d8-19a6262f60c4                                                                     │
│  Agent: Newsletter Orchestrator                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 68999b2a-0f1c-4867-9d88-3150148c6820                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Confirmation string from save_json that the file is saved.                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Confirmation string from save_json that the file is saved.
